In [5]:
import xarray as xr
import dscim
import yaml
from dscim.menu.simple_storage import Climate, EconVars, StackedDamages
import pandas as pd
import numpy as np
from xarray.testing import assert_allclose
from xarray.testing import assert_equal
from itertools import product
from pathlib import Path

import dask
import logging
import subprocess
from subprocess import CalledProcessError
from abc import ABC, abstractmethod
from dscim.descriptors import cachedproperty
from dscim.menu.decorators import save
from dscim.utils.utils import (
    model_outputs,
    compute_damages,
    c_equivalence,
    power,
    quantile_weight_quantilereg,
    extrapolate,
)

In [59]:
# Config vs function params:
# Config should have all paths and parameters that are messy

# function params should be all of the simple parameters that catually want to be changed by EPA

def epa_scc(sector = "CAMEL_m1_c0.20",
            domestic = False,
            eta = 2.0,
            rho = 0.0,
            pulse_year = 2020,
            discount_type = "euler_ramsey",
            menu_option = "risk_aversion",
            gases = ['CO2_Fossil', 'CH4', 'N2O'],
            weitzman_parameters = [0.1, 0.5],
            fair_aggregation = ["mean"]):
    
    USER = "liruixue"
    master = f"/home/{USER}/repos/dscim-epa/damage_fun_runs/master-CAMEL_m1_c0.20.yaml"
    
    with open(master, "r") as stream:
        conf = yaml.safe_load(stream)
        
    if domestic:
        econ = EconVars(
            path_econ=f"{conf['rffdata']['socioec_output']}/rff_USA_socioeconomics.nc4"
        )
    else:
        econ = EconVars(
            path_econ=f"{conf['rffdata']['socioec_output']}/rff_global_socioeconomics.nc4"
        )
    
    conf["global_parameters"] = {'fair_aggregation': fair_aggregation,
     'subset_dict': {'ssp': ssps},
     'weitzman_parameter': weitzman_parameters,
     'save_files': []}

    MENU_OPTIONS = {
        "adding_up": dscim.menu.baseline.Baseline,
        "risk_aversion": dscim.menu.risk_aversion.RiskAversionRecipe,
        "equity": dscim.menu.equity.EquityRecipe,
    }
    print(sector)
    add_kwargs = {
        "econ_vars": econ,
        "climate_vars": Climate(**conf["rff_climate"], pulse_year=pulse_year),
        "formula": conf["sectors"][sector if not domestic else sector[:-4]]["formula"],
        "discounting_type": discount_type,
        "sector": sector,
        "ce_path": None,
        "save_path": None,
        "eta": eta,
        "rho": rho,
        "damage_function_path": Path(conf['paths']['rff_damage_function_library']) / sector / str(2020),
        "ecs_mask_path": None,
        "ecs_mask_name": None,
        "fair_dims":[],
    }

    kwargs = conf["global_parameters"].copy()
    for k, v in add_kwargs.items():
        assert (
            k not in kwargs.keys()
        ), f"{k} already set in config. Please check `global_parameters`."
        kwargs.update({k: v})

    menu_item = MENU_OPTIONS[menu_option](**kwargs)
    menu_item.order_plate("scc")
    uncollapsed_scc = menu_item.uncollapsed_sccs
    
    return(menu_item)
    
# This represents the full gamut of scc runs when run default
def epa_sccs(sectors =["CAMEL_m1_c0.20"],
             etas_rhos = [[1.016010255, 9.149608e-05],
               [1.244459066, 0.00197263997],
               [1.421158116, 0.00461878399]],
             risk_combos = [['risk_aversion', 'euler_ramsey']],
             pulse_years = [2020,2030,2040,2050,2060,2070,2080],
             gases = ['CO2_Fossil', 'CH4', 'N2O'],
             weitzman_parameters = [0.5],
             fair_aggregation = ["mean"]):


    for j, sector in product(risk_combos, sectors):
        all_arrays_uscc = []
        all_arrays_gcnp = []
        discount_type= j[1]
        menu_option = j[0]
        for i, pulse_year in product(etas_rhos, pulse_years):
            eta = i[0]
            rho = i[1]
            df_single = epa_scc(sector = sector,
                                discount_type = discount_type,
                                menu_option = menu_option,
                                eta = eta,
                                rho = rho,
                                pulse_year = pulse_year,
                                gases = gases,
                                weitzman_parameters = weitzman_parameters,
                                fair_aggregation = fair_aggregation)

            df_scc = df_single.uncollapsed_sccs.assign_coords(eta_rhos =  str(eta) + "_" + str(rho), menu_option = menu_option, pulse_year = pulse_year)
            df_scc_expanded = df_scc.expand_dims(['eta_rhos','menu_option','pulse_year'])
            df_scc_expanded.name = "uncollapsed_sccs"
            df_scc_expanded = df_scc_expanded.to_dataset()
            all_arrays_uscc = all_arrays_uscc + [df_scc_expanded]

            df_gcnp = df_single.global_consumption_no_pulse.assign_coords(eta_rhos =  str(eta) + "_" + str(rho), menu_option = menu_option, pulse_year = pulse_year)
            df_gcnp_expanded = df_gcnp.expand_dims(['eta_rhos','menu_option','pulse_year'])
            df_gcnp_expanded.name = "global_consumption_no_pulse"
            df_gcnp_expanded = df_gcnp_expanded.to_dataset()
            all_arrays_gcnp = all_arrays_gcnp + [df_gcnp_expanded]

        df_full_scc = xr.combine_by_coords(all_arrays_uscc)
        df_full_scc.to_netcdf(Path("/home/liruixue/replication_newcode/") / sector / ("full_order_uncollapsed_sccs_" + menu_option + ".nc4"))    

        df_full_gcnp = xr.combine_by_coords(all_arrays_gcnp)
        df_full_gcnp.to_netcdf(Path("/home/liruixue/replication_newcode/") / sector / ("full_order_global_consumption_no_pulse_" + menu_option + ".nc4"))    




FileNotFoundError: [Errno 2] No such file or directory: 'generated_conf.yaml'

In [60]:
sector = ['CAMEL_m1_c0.20'] # Default
etas_rhos = [[1.016010255, 9.149608e-05],
             [1.244459066, 0.00197263997],
             [1.421158116, 0.00461878399]] # Default
risk_combos = [['risk_aversion', 'euler_ramsey']] # Default
gases = ['CO2_Fossil', 'CH4', 'N2O'] # Default
weitzman_parameters = [0.5] # Default
test = epa_sccs(sector,
                etas_rhos,
                risk_combos,
                pulse_years=[2020,2030,2040,2050,2060,2070,2080])

CAMEL_m1_c0.20

 Executing 
        Running risk_aversion
        sector: CAMEL_m1_c0.20
        discounting: euler_ramsey
        eta: 1.016010255
        rho: 9.149608e-05
        
Processing damage functions ...
Subsetting on {} keys.
Global consumption found up to 2300.
Processing SCC calculation ...
Subsetting on {} keys.
Results available: None
CAMEL_m1_c0.20

 Executing 
        Running risk_aversion
        sector: CAMEL_m1_c0.20
        discounting: euler_ramsey
        eta: 1.016010255
        rho: 9.149608e-05
        
Processing damage functions ...
Subsetting on {} keys.
Global consumption found up to 2300.
Processing SCC calculation ...
Subsetting on {} keys.
Results available: None
CAMEL_m1_c0.20

 Executing 
        Running risk_aversion
        sector: CAMEL_m1_c0.20
        discounting: euler_ramsey
        eta: 1.016010255
        rho: 9.149608e-05
        
Processing damage functions ...
Subsetting on {} keys.
Global consumption found up to 2300.
Processing SCC calc

In [58]:
sector = "CAMEL_m1_c0.20"
domestic = False
eta = 2.0
rho = 0.0
pulse_year = 2020
discount_type = "euler_ramsey"
menu_option = "risk_aversion"
gases = ['CO2_Fossil', 'CH4', 'N2O']
weitzman_parameters = [0.1, 0.5]
fair_aggregation = ["mean"]
ssps = []

USER = "liruixue"
master = f"/home/{USER}/repos/dscim-epa/damage_fun_runs/master-CAMEL_m1_c0.20.yaml"

with open(master, "r") as stream:
    conf = yaml.safe_load(stream)

if domestic:
    econ = EconVars(
        path_econ=f"{conf['rffdata']['socioec_output']}/rff_USA_socioeconomics.nc4"
    )
else:
    econ = EconVars(
        path_econ=f"{conf['rffdata']['socioec_output']}/rff_global_socioeconomics.nc4"
    )

conf["global_parameters"] = {'fair_aggregation': fair_aggregation,
 'subset_dict': {'ssp': ssps},
 'weitzman_parameter': weitzman_parameters,
 'save_files': []}

conf['global_parameters']['fair_aggregation'] 

fair_dims = []
MENU_OPTIONS = {
    "adding_up": dscim.menu.baseline.Baseline,
    "risk_aversion": dscim.menu.risk_aversion.RiskAversionRecipe,
    "equity": dscim.menu.equity.EquityRecipe,
}
print(sector)
add_kwargs = {
    "econ_vars": econ,
    "climate_vars": Climate(**conf["rff_climate"], pulse_year=pulse_year),
    "formula": conf["sectors"][sector if not domestic else sector[:-4]]["formula"],
    "discounting_type": discount_type,
    "sector": sector,
    "ce_path": None,
    "save_path": None,
    "eta": eta,
    "rho": rho,
    "damage_function_path": Path(conf['paths']['rff_damage_function_library']) / sector / str(2020),
    "ecs_mask_path": None,
    "ecs_mask_name": None,
    "fair_dims":[]
}

kwargs = conf["global_parameters"].copy()
for k, v in add_kwargs.items():
    assert (
        k not in kwargs.keys()
    ), f"{k} already set in config. Please check `global_parameters`."
    kwargs.update({k: v})

menu_item = MENU_OPTIONS[menu_option](**kwargs)
menu_item.order_plate("scc")
uncollapsed_scc = menu_item.uncollapsed_sccs

CAMEL_m1_c0.20

 Executing 
        Running risk_aversion
        sector: CAMEL_m1_c0.20
        discounting: euler_ramsey
        eta: 2.0
        rho: 0.0
        
Processing damage functions ...
Subsetting on {} keys.
Global consumption found up to 2300.
Processing SCC calculation ...
Subsetting on {} keys.
Results available: None


In [61]:
xr.open_zarr('/shares/gcp/integration/float32/sectoral_ir_damages/mortality_data/impacts-darwin-montecarlo-damages-v1.zarr')

,Array,Chunk
Bytes,161.83 GiB,27.90 MiB
Shape,"(33, 15, 5, 2, 2, 90, 24378)","(1, 15, 1, 1, 1, 10, 24378)"
Count,5941 Tasks,5940 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,161.83 GiB,27.90 MiB
Shape,"(33, 15, 5, 2, 2, 90, 24378)","(1, 15, 1, 1, 1, 10, 24378)"
Count,5941 Tasks,5940 Chunks
Type,float64,numpy.ndarray


In [107]:
test1 = (xr.open_dataset("/home/liruixue/replication_newcode/full_order_uncollapsed_sccseuler_ramsey.nc4")
         .sel(menu_option = 'risk_aversion', weitzman_parameter = '0.5',discount_type = 'euler_ramsey', drop = True)
         .to_array()
        .sel(variable = "uncollapsed_sccs"))
test1_compare = (xr.open_dataset("/home/liruixue/replication_newcode/compare/RFF-SP_allgas_risk_averse_ramsey_sccs_domesticFalse_unmasked_None_Apr272022.nc")
                 .sel(discount_type = "Ramsey", version = "RFF-SP-v2", sector = 'CAMEL',pulse_year = 2020,weitzman = '0.5', drop = True)
                 .to_array()
                .sel(variable = "scc")
                .drop_vars('menu_option'))

a, b = xr.align(test1, test1_compare, join="inner")
c = (a - b) / b
d = np.abs(c.variable.values.flatten())


np.testing.assert_allclose(a,b,rtol = 1e-08)

AssertionError: 
Not equal to tolerance rtol=1e-08, atol=0

Mismatched elements: 15 / 150000 (0.01%)
Max absolute difference: 1.84618402e-05
Max relative difference: 7.52650979e-08
 x: array([[[[[ 1.797611e+02],
          [ 8.083851e+02],
          [ 4.570898e+04]],...
 y: array([[[[[ 1.797611e+02],
          [ 8.083851e+02],
          [ 4.570898e+04]],...

<xarray.Dataset>
Dimensions:             (discount_type: 1, runid: 10000, gas: 3, simulation: 1, weitzman_parameter: 2, fair_aggregation: 1, eta_rhos: 5, menu_option: 1)
Coordinates:
  * discount_type       (discount_type) object 'euler_ramsey'
  * runid               (runid) int64 1 2 3 4 5 6 ... 9996 9997 9998 9999 10000
  * gas                 (gas) object 'CO2_Fossil' 'CH4' 'N2O'
  * simulation          (simulation) int64 1
  * weitzman_parameter  (weitzman_parameter) object '0.1' '0.5'
  * fair_aggregation    (fair_aggregation) object 'uncollapsed'
  * eta_rhos            (eta_rhos) object '1.016010255_9.149608e-05' ... '2.0...
  * menu_option         (menu_option) object 'risk_aversion'
Data variables:
    uncollapsed_sccs    (eta_rhos, menu_option, fair_aggregation, weitzman_parameter, discount_type, runid, gas, simulation) float64 ...

In [100]:
xr.open_dataset("/home/liruixue/replication_newcode/full_order_uncollapsed_sccsconstant.nc4")

<xarray.Dataset>
Dimensions:             (fair_aggregation: 1, discount_type: 1, runid: 10000, gas: 3, simulation: 1, weitzman_parameter: 2, discrate: 6, eta_rhos: 5, menu_option: 1)
Coordinates:
  * fair_aggregation    (fair_aggregation) object 'uncollapsed'
  * discount_type       (discount_type) object 'constant'
  * runid               (runid) int64 1 2 3 4 5 6 ... 9996 9997 9998 9999 10000
  * gas                 (gas) object 'CO2_Fossil' 'CH4' 'N2O'
  * simulation          (simulation) int64 1
  * weitzman_parameter  (weitzman_parameter) object '0.1' '0.5'
  * discrate            (discrate) float64 0.01 0.015 0.02 0.025 0.03 0.05
  * eta_rhos            (eta_rhos) object '1.016010255_9.149608e-05' ... '2.0...
  * menu_option         (menu_option) object 'risk_aversion'
Data variables:
    uncollapsed_sccs    (eta_rhos, menu_option, discrate, fair_aggregation, weitzman_parameter, discount_type, runid, gas, simulation) float64 ...

In [104]:
test1 = (xr.open_dataset("/home/liruixue/replication_newcode/full_order_uncollapsed_sccs_euler_ramsey.nc4")
         .sel(menu_option = 'adding_up', weitzman_parameter = '0.5',discount_type = 'euler_ramsey', drop = True)
         .to_array()
        .sel(variable = "uncollapsed_sccs"))
test1_compare = (xr.open_dataset("/home/liruixue/replication_newcode/compare/RFF-SP_allgas_risk_neutral_ramsey_sccs_domesticFalse_unmasked_None_Apr272022.nc")
                 .sel(discount_type = "Ramsey", version = "RFF-SP-v2", sector = 'CAMEL',pulse_year = 2020,weitzman = '0.5', drop = True)
                 .to_array()
                .sel(variable = "scc")
                .drop_vars('menu_option'))

a, b = xr.align(test1, test1_compare, join="inner")
c = (a - b) / b
d = np.abs(c.variable.values.flatten())


np.testing.assert_allclose(a,b,rtol = 1e-08)

AssertionError: 
Not equal to tolerance rtol=1e-08, atol=0

Mismatched elements: 22 / 150000 (0.0147%)
Max absolute difference: 5.72018325e-06
Max relative difference: 1.75003765e-07
 x: array([[[[[ 1.370883e+02],
          [ 5.964344e+02],
          [ 3.553115e+04]],...
 y: array([[[[[ 1.370883e+02],
          [ 5.964344e+02],
          [ 3.553115e+04]],...

In [57]:
#xr.open_dataset("/shares/gcp/integration_replication/results/rff/agriculture/2020/risk_aversion_euler_ramsey_eta2.0_rho0.0_damage_function_coefficients.nc4")
t = xr.open_dataset("/home/liruixue/replication_newcode/agriculture/full_order_uncollapsed_sccs_euler_ramsey.nc4").sel(eta_rhos = '2.0_0.0', pulse_year = 2020, menu_option = 'adding_up', drop = True)
t1 = xr.open_dataset("/shares/gcp/integration_replication/results/rff/agriculture/2020/adding_up_euler_ramsey_eta2.0_rho0.0_uncollapsed_sccs.nc4")
a,b = xr.align(t,t1,join ='inner')
np.testing.assert_allclose(a.to_array(),b.to_array(), rtol = 1e-100)


In [51]:
b

<xarray.Dataset>
Dimensions:             (discount_type: 1, runid: 10000, gas: 3, weitzman_parameter: 2, fair_aggregation: 1)
Coordinates:
  * discount_type       (discount_type) object 'euler_ramsey'
  * runid               (runid) int64 1 2 3 4 5 6 ... 9996 9997 9998 9999 10000
  * gas                 (gas) object 'CO2_Fossil' 'CH4' 'N2O'
  * weitzman_parameter  (weitzman_parameter) object '0.1' '0.5'
  * fair_aggregation    (fair_aggregation) object 'uncollapsed'
Data variables:
    uncollapsed_sccs    (fair_aggregation, weitzman_parameter, discount_type, runid, gas) float64 ...
Attributes: (12/49)
    sector_path:                    /shares/gcp/integration/float32/input_dat...
    save_path:                      /shares/gcp/integration_replication/resul...
    gdppc_bottom_code:              39.39265060424805
    subset_dict:                    {'ssp': ['SSP2', 'SSP3', 'SSP4']}
    econ_vars:                      <dscim.menu.simple_storage.EconVars objec...
    climate:                        <dscim.menu.simple_storage.Climate object...
    ...                             ...
    gases:                          ['CO2_Fossil', 'CH4', 'N2O']
    base_period:                    [2001, 2010]
    anomaly_vars:                   ['temperature', 'gmsl']
    path:                           /shares/gcp/integration_replication/input...
    machine:                        battuta
    commit:                         b'0bdcf93'